In [87]:
!pip install datasets
!pip install scipy
!pip install apache_beam
!pip install Wikipedia-API
!pip install requests-futures
!pip install apache-beam[interactive]

Defaulting to user installation because normal site-packages is not writeable
  Using cached dill-0.3.6-py3-none-any.whl (110 kB)
  Attempting uninstall: dill
    Found existing installation: dill 0.3.1.1
    Uninstalling dill-0.3.1.1:
      Successfully uninstalled dill-0.3.1.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
apache-beam 2.45.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.6 which is incompatible.
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 41.5 MB/s eta 0:00:0000:0100:01
Defaulting to user installation because normal site-packages is not writeable
  Using cached dill-0.3.1.1-py3-none-any.whl
  Attempting uninstall: dill
    Found existing installation: dill 0.3.6
    Uninstalling dill-0.3.6:
      Successfully uninstalled dill-0.3.6
ERROR: pip's

In [2]:
import re
#import wikipediaapi
from datasets import load_dataset
import requests

In [3]:

def extract_title(page_url):
    return re.search(r'title=(.*?)&amp;', page_url).group(1)

def gen_request_url_for_2021_views(page_url):

    title = extract_title(page_url)

    url = f'https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/en.wikipedia/all-access/all-agents/{title}/monthly/2021010100/2021123100'

    return url

In [4]:
#dataset = load_dataset("natural_questions", "dev", split="validation", cache_dir="/mnt/scratch/alopardo/natural_questions")
#dataset = load_dataset("natural_questions", 'default', beam_runner='DirectRunner', ignore_verifications=True, cache_dir="/mnt/scratch/alopardo/natural_questions")
ds = Dataset.from_file("natural_questions-train.arrow")

Downloading:   0%|          | 0.00/17.4k [00:00<?, ?B/s]

/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 1.16.0-unknown is an invalid version and will not be supported in a future release
  warnings.warn(
/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 1.1build1 is an invalid version and will not be supported in a future release
  warnings.warn(
/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 0.1.43ubuntu1 is an invalid version and will not be supported in a future release
  warnings.warn(


In [4]:
headers = {
        'User-Agent': 'Antonio Lopardo',
        'From': 'antonio.lopardo@outlook.com'  # This is another valid field
    }

In [9]:
wiki_wiki = wikipediaapi.Wikipedia('en')
responses = [query_for_2021_views(dataset[i]['document']["url"]) for i in range(1000)]

In [5]:
from concurrent.futures import as_completed
from requests_futures.sessions import FuturesSession
from concurrent.futures import ThreadPoolExecutor

with FuturesSession(executor=ThreadPoolExecutor(max_workers=32)) as session:
    futures = [session.get(gen_request_url_for_2021_views(dataset[i]['document']["url"]), headers=headers) for i in range(1000)]
    for future in as_completed(futures):
        resp = future.result()

In [6]:
for future in futures:
        resp = future.result()
        print(resp.json()['url'])

https://httpbin.org/delay/0
https://httpbin.org/delay/1
https://httpbin.org/delay/2
https://httpbin.org/delay/0
https://httpbin.org/delay/1
https://httpbin.org/delay/2
https://httpbin.org/delay/0
https://httpbin.org/delay/1
https://httpbin.org/delay/2
https://httpbin.org/delay/0


In [1]:
import pickle

with open('monthly_views_results_full.pickle', 'rb') as handle:
    results = pickle.load(handle)


In [2]:
results[0]

'{"items":[{"project":"en.wikipedia","article":"The_Walking_Dead_(season_8)","granularity":"monthly","timestamp":"2021010100","access":"all-access","agent":"all-agents","views":26344},{"project":"en.wikipedia","article":"The_Walking_Dead_(season_8)","granularity":"monthly","timestamp":"2021020100","access":"all-access","agent":"all-agents","views":28556},{"project":"en.wikipedia","article":"The_Walking_Dead_(season_8)","granularity":"monthly","timestamp":"2021030100","access":"all-access","agent":"all-agents","views":37578},{"project":"en.wikipedia","article":"The_Walking_Dead_(season_8)","granularity":"monthly","timestamp":"2021040100","access":"all-access","agent":"all-agents","views":30701},{"project":"en.wikipedia","article":"The_Walking_Dead_(season_8)","granularity":"monthly","timestamp":"2021050100","access":"all-access","agent":"all-agents","views":24895},{"project":"en.wikipedia","article":"The_Walking_Dead_(season_8)","granularity":"monthly","timestamp":"2021060100","access":

In [26]:
def get_total_views(result):
    json_results = json.loads(result)
    
    try:
        monthly_views = [item["views"] for item in json_results["items"]]
    except:
        monthly_views = [-1]
    
    return sum(monthly_views)

In [27]:
import json
test = json.loads(results[0])
test["items"][0]["views"]

26344

In [28]:
total_views_per_query = [get_total_views(results[i]) for i in range(len(results))]

In [20]:
import numpy as np
views_per_query_array = np.array(total_views_per_query)

In [88]:
import scipy

percentiles = [scipy.stats.percentileofscore(views_per_query_array, i) for i in views_per_query_array[:100]]

In [85]:
np.percentile(views_per_query_array, 50)

176008.0

In [58]:
wrong_queries_ids = np.where(views_per_query_array == -1)
wrong_queries_ids[0].shape

(611,)

In [32]:
from datasets import Dataset

def extract_title(page_url):
    return re.search(r'title=(.*?)&amp;', page_url).group(1)

def gen_request_url_for_2021_views(page_url):

    title = extract_title(page_url)

    url = f'https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/en.wikipedia/all-access/all-agents/{title}/monthly/2021010100/2021123100'

    return url

headers = {
        'User-Agent': 'Antonio Lopardo',
        'From': 'antonio.lopardo@outlook.com'  # This is another valid field
    }

dataset = Dataset.from_file("/mnt/scratch/alopardo/natural_questions/natural_questions-train.arrow")

In [79]:
import requests

results_fourOfour = []

for i in wrong_queries_ids[0]:
    
    i = int(i)
    
    result = requests.get(gen_request_url_for_2021_views(dataset[i]['document']["url"]), headers=headers)
    
    results_fourOfour.append(result)

In [84]:
requests.get(gen_request_url_for_2021_views(dataset[int(wrong_queries_ids[0][0])]['document']["url"]), headers=headers)

<Response [404]>